<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Performance_Over_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

print_date_time()

gen_output = 0

Run date time (IST): 2025-03-12 17:46:04


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np
from google.colab import data_table

#mypfs = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myPortfolioStocks.csv')
mypps = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')

#mypf = pd.merge(mypfs, mypps, on="Symbol")

#mypf = mypf[~mypf['InPortfolio'].isnull()]

#mypf = mypf[mypf['StarStock'] == 1]

stock_n100 = mypps['Symbol'].unique()

exclude = ['CADILAHC','MMTC', 'ADANITRANS']

#stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']
stock_ids = list(set(list(stock_n100)) - set(exclude))

In [4]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin

start_date = (datetime.date.today() + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

df_perform = pd.DataFrame()

for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    #tmp = pd.DataFrame(web.DataReader(tickers = stock_id, start = start_date, end = end_date))
    tmp = yfin.Ticker(stock_id).history(period='5d', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    tmp['Stock'] = stock_id
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['Change%'] = round((tmp['Close'] - tmp['Prev_Close'])*100/tmp['Prev_Close'],2)
    tmp = tmp.tail(1)
    #df_perform = df_perform.append(tmp)
    df_perform = pd.concat([df_perform, tmp], ignore_index = True)


ERROR:yfinance:$TV18BRDCST.NS: possibly delisted; no price data found  (period=5d)
ERROR:yfinance:$TIPSINDLTD.NS: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$WIPRO.NS: possibly delisted; no price data found  (period=5d)
ERROR:yfinance:$SCHAEFFLER .NS: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


In [11]:
d = 365*5
start_date = (datetime.date.today() + datetime.timedelta(days=-d)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=0)).strftime('%Y-%m-%d')

df_perform = pd.DataFrame()

for stock_id in stock_ids:
    #stock_id = stock_id.upper() + '.NS'

    #df_stock = pd.DataFrame(web.DataReader(tickers = stock_id, start = start_date, end = end_date))
    df_stock = yfin.Ticker(stock_id + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    if df_stock.empty:
        print(f"Warning: No data found for {stock_id}. Skipping...")
        continue  # Skip to the next stock_id

    CMP = round(df_stock['Close'].values[-1],0)
    D1 = round((df_stock['Close'].values[-1] - df_stock['Close'].shift(1).values[-1])*100/df_stock['Close'].shift(1).values[-1],2)
    W1 = round((df_stock['Close'].values[-1] - df_stock['Close'].shift(5).values[-1])*100/df_stock['Close'].shift(5).values[-1],2)
    M1 = round((df_stock['Close'].values[-1] - df_stock['Close'].shift(20).values[-1])*100/df_stock['Close'].shift(20).values[-1],2)
    M3 = round((df_stock['Close'].values[-1] - df_stock['Close'].shift(60).values[-1])*100/df_stock['Close'].shift(60).values[-1],2)
    M6 = round((df_stock['Close'].values[-1] - df_stock['Close'].shift(120).values[-1])*100/df_stock['Close'].shift(120).values[-1],2)
    Y1 = round((df_stock['Close'].values[-1] - df_stock['Close'].shift(240).values[-1])*100/df_stock['Close'].shift(240).values[-1],2)
    Y2 = round(((df_stock['Close'].values[-1]/df_stock['Close'].shift(240*2).values[-1])**.5 - 1)*100,2)
    Y3 = round(((df_stock['Close'].values[-1]/df_stock['Close'].shift(240*3).values[-1])**.33 - 1)*100,2)
    Y5 = round(((df_stock['Close'].values[-1]/df_stock['Close'].shift(240*5).values[-1])**.2 - 1)*100,2)


    tmp = {'Stock':[stock_id],
           'CMP': CMP,
           '1D%' : [D1],
           '1W%' : [W1],
           '1M%' : [M1],
           '3M%' : [M3],
           '6M%' : [M6],
           '1Y%' : [Y1],
           '2Y%' : [Y2],
           '3Y%' : [Y3],
           '5Y%' : [Y5]
           }

    tmp = pd.DataFrame(tmp)

    #df_perform = df_perform.append(tmp)
    df_perform = pd.concat([df_perform, tmp], ignore_index = True)

mypps['Stock'] = mypps['Symbol']
cols = ['Stock', 'Category', 'InFolio', 'Conviction']
df_tmp = mypps[cols]
df_perform = pd.merge(df_tmp, df_perform, on="Stock")
cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
myscreen_df = myscreen_df[cols]
myscreen_df['Stock'] = myscreen_df['Symbol']
df_perform = pd.merge(myscreen_df, df_perform, on="Stock")
df_perform['Curr_PE'] = round(df_perform['CMP']/df_perform['EPS'],1)
df_perform['Dev%_PE'] = round((df_perform['Curr_PE'] - df_perform['MedPE'])*100/df_perform['MedPE'],2)
df_perform['Conviction'] = df_perform['Conviction'] + '-' + df_perform['CapType']
cols = ['Symbol', 'Category', 'InFolio', 'Conviction', 'Dev%_PE', '1W%','1M%','3M%','6M%','1Y%','2Y%','3Y%','5Y%'	]
df_perform = df_perform[cols]

ERROR:yfinance:$TIPSINDLTD.NS: possibly delisted; no timezone found


ERROR:yfinance:$WIPRO.NS: possibly delisted; no price data found  (1d 1926-04-06 -> 2025-03-12)


ERROR:yfinance:$SCHAEFFLER .NS: possibly delisted; no timezone found


In [12]:
# all stocks
data_table.DataTable(df_perform, include_index=False)

,Symbol,Category,Dev%_PE,InFolio,Conviction,1W%,1M%,3M%,6M%,1Y%,2Y%,3Y%,5Y%
0,RELIANCE,REFINERIES,-6.89,1.0,X-LC,6.93,1.80,0.94,-14.47,-13.02,8.66,2.75,13.37
1,TCS,IT,-16.51,1.0,X-LC,-1.15,-11.54,-18.80,-17.99,-9.05,6.24,0.67,14.40
2,HDFCBANK,BANKS,-7.24,1.0,X-LC,1.25,0.57,-6.66,0.16,20.21,4.40,6.54,13.95
3,BHARTIARTL,TELECOM,-42.18,NaN,H-LC,1.52,-3.18,1.71,-1.32,33.65,48.85,30.70,25.11
4,ICICIBANK,BANKS,-15.24,NaN,X-LC,2.32,-0.70,-6.73,-3.72,15.07,20.58,18.75,30.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,SURYODAY,BANKS,-51.15,1.0,H-SC,-9.25,-11.32,-31.88,-44.73,-39.36,2.45,-11.83,NaN
225,5PAISA,FINANCE,-56.00,1.0,H-SC,-0.47,-10.96,-30.99,-36.41,-30.83,9.20,-3.07,17.29
226,VALIANTORG,CHEMICALS,-215.40,1.0,H-SC,-9.03,-12.52,-26.61,-42.49,-41.70,-24.02,-37.22,NaN
227,ASIANTILES,CERAMICS,279.65,1.0,M-SC,-3.27,-19.10,-36.42,-45.87,-25.99,13.72,-21.11,-12.25


In [13]:
# portfolio stocks
tmp_df = df_perform[(df_perform['InFolio'] == 1) ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Category,Dev%_PE,InFolio,Conviction,1W%,1M%,3M%,6M%,1Y%,2Y%,3Y%,5Y%
0,RELIANCE,REFINERIES,-6.89,1.0,X-LC,6.93,1.80,0.94,-14.47,-13.02,8.66,2.75,13.37
1,TCS,IT,-16.51,1.0,X-LC,-1.15,-11.54,-18.80,-17.99,-9.05,6.24,0.67,14.40
2,HDFCBANK,BANKS,-7.24,1.0,X-LC,1.25,0.57,-6.66,0.16,20.21,4.40,6.54,13.95
5,INFY,IT,-14.21,1.0,X-LC,-7.05,-15.18,-19.52,-15.06,8.15,8.31,-0.71,20.83
6,SBIN,BANKS,-26.83,1.0,M-LC,-1.00,-1.10,-14.99,-8.47,-1.54,19.62,13.56,34.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,SURYODAY,BANKS,-51.15,1.0,H-SC,-9.25,-11.32,-31.88,-44.73,-39.36,2.45,-11.83,NaN
225,5PAISA,FINANCE,-56.00,1.0,H-SC,-0.47,-10.96,-30.99,-36.41,-30.83,9.20,-3.07,17.29
226,VALIANTORG,CHEMICALS,-215.40,1.0,H-SC,-9.03,-12.52,-26.61,-42.49,-41.70,-24.02,-37.22,NaN
227,ASIANTILES,CERAMICS,279.65,1.0,M-SC,-3.27,-19.10,-36.42,-45.87,-25.99,13.72,-21.11,-12.25


In [15]:
# high conviction prospects
tmp_df = df_perform[(df_perform['InFolio'] != 1) & (df_perform['Conviction'].isin(['X-LC','H-LC']))]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Category,Dev%_PE,InFolio,Conviction,1W%,1M%,3M%,6M%,1Y%,2Y%,3Y%,5Y%
3,BHARTIARTL,TELECOM,-42.18,NaN,H-LC,1.52,-3.18,1.71,-1.32,33.65,48.85,30.70,25.11
4,ICICIBANK,BANKS,-15.24,NaN,X-LC,2.32,-0.70,-6.73,-3.72,15.07,20.58,18.75,30.65
8,BAJFINANCE,FINANCE,-16.16,0.0,X-LC,2.24,3.58,18.62,11.78,26.12,23.47,5.40,31.84
9,ITC,FMCG,14.70,NaN,H-LC,1.81,0.15,-10.78,-17.58,-0.53,7.02,19.00,25.59
11,LT,ENGINEERING,4.79,NaN,X-LC,-1.42,-1.42,-16.12,-13.30,-11.03,23.06,23.27,31.95
13,SUNPHARMA,PHARMA,15.22,NaN,X-LC,5.97,-1.35,-5.71,-8.62,5.18,31.81,22.36,31.27
14,MARUTI,AUTO,-33.26,NaN,X-LC,0.20,-8.52,4.79,-5.76,-4.76,19.60,16.74,19.21
15,KOTAKBANK,BANKS,-36.62,0.0,X-LC,2.56,3.33,11.13,5.91,11.73,7.07,3.71,10.88
16,M&M,AUTO,8.97,NaN,H-LC,-2.69,-14.02,-12.76,-5.15,42.26,53.47,46.53,46.63
17,ULTRACEMCO,CEMENT,44.26,NaN,X-LC,0.54,-6.90,-10.65,-9.52,9.31,18.10,16.51,24.89


In [9]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 124.0 seconds
